In [ ]:
# ===============================
# CÉLULA 0 — CHECK DE GPU
# ===============================

import torch, sys

print("Torch:", torch.__version__)
print("CUDA disponível:", torch.cuda.is_available())

if not torch.cuda.is_available():
    raise RuntimeError(
        "❌ GPU NÃO ATIVA\n"
        "Ative em: Ambiente de execução → Alterar tipo → GPU\n"
        "Depois REINICIE o runtime."
    )

print("GPU detectada:", torch.cuda.get_device_name(0))
print("✅ GPU OK — pode continuar")


In [ ]:
# ===============================
# CÉLULA 1 — HUGGING FACE LOGIN
# ===============================

import os
from huggingface_hub import login

if "HUGGINGFACE_TOKEN" in os.environ:
    login(token=os.environ["HUGGINGFACE_TOKEN"])
    print("✅ Hugging Face autenticado com token")
else:
    print("⚠️ HUGGINGFACE_TOKEN não encontrado")
    print("➡️ Acesso público será usado (ok para Qwen 7B)")


In [ ]:
# ===============================
# CÉLULA 2 — SETUP
# ===============================

import os, sys, subprocess

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

def pip_install(args):
    subprocess.check_call([sys.executable, "-m", "pip"] + args.split())

def pip_uninstall(pkg):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", pkg])
    except:
        pass

# NÃO reinstala torch (Colab já vem com CUDA correto)
pip_install("install -q -U transformers accelerate huggingface_hub")

pip_uninstall("bitsandbytes")
pip_install("install -q -U bitsandbytes")

print("✅ Ambiente configurado")


In [ ]:
# ===============================
# CÉLULA 3 — MODELO QWEN 2.5 7B
# ===============================

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_ID = "Qwen/Qwen2.5-7B-Instruct"

# Tokenizer (leve, não ocupa VRAM)
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    use_fast=True,
    trust_remote_code=True
)

# Quantização 4-bit otimizada para T4
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# 🚫 NUNCA usar device_map="auto" no Colab Free
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map={"": 0},          # força TUDO na GPU
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,      # evita estouro de RAM
    trust_remote_code=True,
)

model.eval()

print("✅ Qwen2.5-7B carregado com sucesso")
print("VRAM usada (GB):", round(torch.cuda.memory_allocated() / 1024**3, 2))


In [ ]:
# ===============================
# CÉLULA 4 — INFERÊNCIA SEGURA
# ===============================

def ask_qwen(
    prompt,
    max_new_tokens=700,
    temperature=0.2,
    top_p=0.95,
    max_prompt_tokens=6000,  # CONTROLE REAL
    show_usage=False
):
    messages = [
        {"role": "system", "content": "Você é um especialista em Engenharia de Software e DevOps."},
        {"role": "user", "content": prompt},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        truncation=True,
        max_length=max_prompt_tokens,
        return_tensors="pt"
    ).to(model.device)

    if show_usage:
        print("Prompt tokens:", input_ids.shape[-1])

    with torch.inference_mode():
        output = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id,
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded.split(messages[-1]["content"])[-1].strip()


In [ ]:
# ===============================
# CÉLULA 5 — TESTE
# ===============================

resp = ask_qwen(
    "Explique em 3 linhas o que é GitFlow.",
    show_usage=True
)

print(resp)


In [ ]:
# ===============================
# CÉLULA 6 — COLETA GIT
# ===============================

import subprocess, os
from collections import Counter
from IPython.display import Markdown, display

def git(cmd, cwd):
    return subprocess.check_output(cmd, cwd=cwd, shell=True).decode().strip()

repo_url = "https://github.com/unclecode/crawl4ai.git"
repo_dir = "/content/repo"

if not os.path.exists(repo_dir):
    subprocess.check_call(["git", "clone", repo_url, repo_dir])

branches = git("git branch -r | grep -v HEAD", repo_dir).splitlines()
branches = [b.replace("origin/", "").strip() for b in branches]

tags = git("git tag --list", repo_dir).splitlines()

bucket = Counter(b.split('/')[0] if '/' in b else 'root' for b in branches)

display(Markdown(f"- **Branches:** {len(branches)}"))
display(Markdown(f"- **Tags:** {len(tags)}"))
display(Markdown(f"```{bucket}```"))


In [ ]:
# ===============================
# CÉLULA 7 — ANÁLISE
# ===============================

prompt = f"""
Analise o modelo de branching com base nos dados reais.

Branches totais: {len(branches)}
Distribuição:
{bucket}

Amostra:
{sorted(branches)[:40]}

Classifique:
A) GitFlow
B) GitHub Flow
C) Trunk-Based Development

Justifique com evidências reais.
"""

resp = ask_qwen(prompt)
display(Markdown("## 🌿 Branching Model"))
display(Markdown(resp))


In [ ]:
# ===============================
# CÉLULA 8 — RELEASES
# ===============================

prompt = f"""
Analise a estratégia de releases com base nas tags abaixo.

Total de tags: {len(tags)}
Amostra de tags:
{tags[-20:]}

Classifique:
- Rapid Release
- Release Train
- LTS + Current
- Ad-hoc
"""

print(ask_qwen(prompt))
